# Профилилирование и оптимизация выполнения кода

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. "Оптимизация выполнения кода, векторизация, Numba"
* IPython Cookbook, Second Edition (2018), глава 4
* https://ipython-books.github.io/43-profiling-your-code-line-by-line-with-line_profiler/

## Задачи для совместного разбора

In [2]:
!pip install line_profiler
# !pip install --user numpy==1.20

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 673.6/673.6 KB 11.3 MB/s eta 0:00:00


In [3]:
import numpy as np

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [4]:
A = np.random.randint(0,1001,1000000)
A[:7], len(A)

(array([778, 174, 876, 617, 246, 766, 199]), 1000000)

In [5]:
# тупо
def f(A):
    s, c = 0, 0
    for a in A:
        b = a + 100
        s += b
        c += 1
    return s/c

In [6]:
%%time
f(A)

CPU times: user 347 ms, sys: 435 µs, total: 347 ms
Wall time: 353 ms


599.943123

In [7]:
# исправлено
def f(A):
    return sum(A)/len(A) + 100

In [8]:
%%timeit
f(A)

174 ms ± 94.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
f(A)

599.943123

In [10]:
%%timeit
f(A)

171 ms ± 85.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
A.min(), A.max()

(0, 1000)

In [12]:
%load_ext line_profiler

In [13]:
def slow(A):
    s, c = 0, 0
    for a in A:
        b = a + 100
        s += b
        c += 1
    return s/c

In [100]:
%lprun -f slow slow(A)

In [15]:
%%timeit
B = A + 100
B.mean()

2.65 ms ± 97.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [16]:
import pandas as pd
import string

N = 2_000_000
df = pd.DataFrame(np.random.randn(N, 4), columns=[f"col{i}" for i in range(4)])
df["key"] = np.random.choice(list(string.ascii_letters.lower()), N, replace=True)
df.head(2)

,col0,col1,col2,col3,key
0,-1.26342,1.707213,-0.863306,-1.112995,s
1,0.06061,0.590105,2.058300,-0.669511,y


In [17]:
%%timeit
df[df['key'].isin(set(string.ascii_letters[:5]))]

134 ms ± 29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit
df[df['key'].isin(['a', 'b', 'c', 'd', 'e'])]

134 ms ± 23.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%%timeit
df[df['key'] <= 'e']

166 ms ± 42 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
df[df['key'].isin(list(string.ascii_letters.lower())[:5])]

,col0,col1,col2,col3,key
2,1.162012,1.257318,0.999725,-0.520977,b
13,-0.719597,0.699139,-2.910598,-0.371704,e
22,-0.088064,-0.744626,0.636114,-1.537266,b
40,0.269133,-0.760796,-0.745792,-1.023436,b
41,0.500429,-2.564897,0.682327,-0.426440,b
...,...,...,...,...,...
1999974,-0.940668,-0.928623,0.194177,-0.366606,e
1999975,0.360805,-0.429494,-0.432076,0.271763,e
1999982,0.506294,2.006006,0.824587,-1.606037,c
1999988,-1.967922,-1.619738,-1.249026,0.945247,e


In [22]:
df[df['key'].isin(list(string.ascii_lowercase[:5]))]

,col0,col1,col2,col3,key
2,1.162012,1.257318,0.999725,-0.520977,b
13,-0.719597,0.699139,-2.910598,-0.371704,e
22,-0.088064,-0.744626,0.636114,-1.537266,b
40,0.269133,-0.760796,-0.745792,-1.023436,b
41,0.500429,-2.564897,0.682327,-0.426440,b
...,...,...,...,...,...
1999974,-0.940668,-0.928623,0.194177,-0.366606,e
1999975,0.360805,-0.429494,-0.432076,0.271763,e
1999982,0.506294,2.006006,0.824587,-1.606037,c
1999988,-1.967922,-1.619738,-1.249026,0.945247,e


## Лабораторная работа 1

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy` и `pandas`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy` или структур `pandas` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

В файлах `recipes_sample.csv` и `reviews_sample.csv` находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

In [175]:
recipes = pd.read_csv('recipes_sample.csv', parse_dates=['submitted'])
reviews = pd.read_csv('reviews_sample.csv')

In [24]:
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [25]:
reviews.head()

,Unnamed: 0,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


## Измерение времени выполнения кода

Создайте версию таблицы, содержащие строки строки для рецептов, которые были добавлены в 2010 году.

Реализуйте несколько вариантов функции подсчета средней длины полного описания рецепта для рецептов, добавленных в 2010 году. Полным описанием рецепта называется строка, полученная путем конкатенации названия и описания рецепта через пробел.

In [26]:
df1 = recipes[recipes['submitted'].dt.year == 2010]
df1.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
52,just peachy cobbler,437637,70,1085867,2010-09-17,10.0,all i can say is yummmmmm . . . a simple to ma...,10.0
68,the heat spicy party mix,437219,95,1682162,2010-09-13,NaN,a spicy chex mix that will really warm your gu...,11.0
81,iowa state fair sweet dough caramel cinnamon ...,435816,80,17803,2010-08-24,29.0,this was the winning entry at the 2010 iowa st...,NaN
104,1 minute blueberries cream,428566,2,1375473,2010-06-04,4.0,i was craving blueberry tonight but wanted non...,NaN
146,2 2 2 diet mocha,416599,5,789314,2010-03-15,5.0,"while trying to come up with a satisfying ""sna...",7.0


№1\.1 С использованием метода `DataFrame.iterrows` таблицы:

- функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
- вычисление полного описания рецепта осуществляется внутри цикла по `iterrows` для каждой строки по отдельности.

In [27]:
def get_mean_len_A(df: pd.DataFrame) -> float:
    s = k = 0
    for i in df.iterrows():
        s += len(i[1][0]) + len(i[1][6]) + 1
        k += 1
    return s/k

get_mean_len_A(df1)

265.501300390117

№1\.2. С использованием метода `DataFrame.apply` таблицы:

- функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
- вызываете метод apply у таблицы; в качестве аргумента передаете функцию, которая возвращает длину полного описания для каждой строки;
    
- считаете среднюю длину описаний, вызвав соответствующий метод серии.

In [32]:
def get_mean_len_B(df: pd.DataFrame) -> float:
    ser = df.apply(lambda x: len(x['name']) + len(x['description']) + 1, axis=1)
    return ser.mean()

get_mean_len_B(df1)

265.501300390117

№1\.3. С использованием векторизованных методов серий `pd.Series`:

- функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
- при помощи векторизованной операции сложения получаете столбец с полным описанием;
    
- считаете длину каждого элемента столбца с полным описанием, воспользовавшись соответствующим строковым методом аксессора `.str`;
    
- считаете среднюю длину описаний, вызвав соответствующий метод серии.

In [106]:
def get_mean_len_C(df):
    vec_desc = np.vectorize(lambda x,y: ' '.join([x,y]))
    return pd.Series(vec_desc(df['name'],df['description'])).str.len().mean()

get_mean_len_C(df1)

265.501300390117

№1.4 Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций при помощи магических команд `time` и `timeit`.

In [95]:
%%timeit
get_mean_len_A(df1)

77.5 ms ± 11.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [96]:
%%timeit
get_mean_len_B(df1)

26.1 ms ± 982 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [108]:
%%timeit
get_mean_len_C(df1)

9.4 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## **Вывод**
* Apply уменьшает время в $\sim$ 3 раза
* Векторизация уменьшает время в $\sim$ 8 раза

## Анализ пошагового выполнения кода 

Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. 

In [176]:
import re


def get_word_reviews_count(df):
    word_reviews = {}
    for review_id, row in df.dropna(subset=["review"]).iterrows():
        review = row["review"]
        words = re.sub(r"[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            if word.lower() not in word_reviews:
                word_reviews[word.lower()] = set()
            word_reviews[word.lower()].add(review_id)
    word_reviews_count = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        review = row["review"]
        words = re.sub(r"[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            word_reviews_count[word.lower()] = len(word_reviews[word.lower()])
    return word_reviews_count

In [177]:
get_by_bad = get_word_reviews_count(reviews)

In [178]:
get_by_bad

{'last': 4517,
 'week': 1489,
 'whole': 5540,
 'sides': 435,
 'of': 61867,
 'frozen': 2722,
 'salmon': 819,
 'fillet': 86,
 'was': 56972,
 'on': 28791,
 'sale': 255,
 'in': 43940,
 'my': 44544,
 'local': 565,
 'supermarket': 93,
 'so': 39441,
 'i': 101329,
 'bought': 1490,
 'tons': 184,
 'okay': 717,
 'only': 13679,
 '': 89125,
 'but': 36936,
 'total': 557,
 'weight': 290,
 'over': 8762,
 'pounds': 275,
 'this': 83593,
 'recipe': 54531,
 'is': 41236,
 'perfect': 8643,
 'for': 75829,
 'even': 8881,
 'though': 4791,
 'it': 73971,
 'calls': 513,
 'steaks': 434,
 'cut': 6416,
 'up': 14352,
 'the': 95894,
 'into': 6364,
 'individual': 304,
 'portions': 209,
 'and': 97007,
 'followed': 5450,
 'instructions': 971,
 'exactly': 4678,
 'im': 7768,
 'one': 15973,
 'those': 2408,
 'food': 3473,
 'combining': 82,
 'diets': 45,
 'left': 4958,
 'out': 22223,
 'white': 3493,
 'wine': 1580,
 'added': 19387,
 'just': 23483,
 'a': 84192,
 'dash': 617,
 'vinegar': 1641,
 'instead': 11221,
 'little': 14634

№2.1 Найдите узкие места в коде, проанализировав код функции по шагам, используя профайлер. Сохраните результаты работы профайлера в отдельную текстовую ячейку. Выпишите (словами), что в имеющемся коде реализовано неоптимально. 

In [116]:
%lprun -T lprof0 -f get_word_reviews_count get_word_reviews_count(reviews.head(1000))


*** Profile printout saved to text file 'lprof0'. 


In [126]:
f = open('lprof0')
for row in f.readlines():
  print(row)
f.close()

Timer unit: 1e-09 s



Total time: 0.345077 s

File: <ipython-input-110-f37ec2aa0f5d>

Function: get_word_reviews_count at line 4



Line #      Hits         Time  Per Hit   % Time  Line Contents


     4                                           def get_word_reviews_count(df):

     5         1       1192.0   1192.0      0.0      word_reviews = {}

     6      1000   84908893.0  84908.9     24.6      for review_id, row in df.dropna(subset=["review"]).iterrows():

     7      1000   11788551.0  11788.6      3.4          review = row["review"]

     8      1000   15310680.0  15310.7      4.4          words = re.sub(r"[^A-Za-z\s]", "", review).split(" ")

     9     54065   12013038.0    222.2      3.5          for word in words:

    10     49460   22421033.0    453.3      6.5              if word.lower() not in word_reviews:

    11      4605    4567278.0    991.8      1.3                  word_reviews[word.lower()] = set()

    12     54065   34626987.0    640.5     10.0              

№2.2  Оптимизируйте функцию и добейтесь значительного (как минимум, в 5 раз) прироста в скорости выполнения. Для демонстрации результата измерьте скорость выполнения оригинальной функции и функции, написанной вами.

In [233]:
def get_word_reviews_count_opt(df):
    word_reviews_count = {}
    df1 = df['review'].dropna()
    for row in df1:
        #print(row)
        words = re.sub("[^A-Za-z\s]", "", row.lower()).split(" ")
        for word in set(words):
            word_reviews_count[word] = word_reviews_count.get(word,0)+1
    return word_reviews_count

In [234]:
lprun -f get_word_reviews_count_opt get_word_reviews_count_opt(reviews.head(100))

In [235]:
get_by_good = get_word_reviews_count_opt(reviews)

In [236]:
get_by_good

{'': 89125,
 'followed': 5450,
 'individual': 304,
 'combining': 82,
 'last': 4517,
 'supermarket': 93,
 'total': 557,
 'tons': 184,
 'of': 61867,
 'bit': 9864,
 'one': 15973,
 'i': 101329,
 'taste': 10551,
 'calls': 513,
 'bought': 1490,
 'my': 44544,
 'steaks': 434,
 'week': 1489,
 'just': 23483,
 'dash': 617,
 'recipe': 54531,
 'diets': 45,
 'instead': 11221,
 'today': 1992,
 'perfect': 8643,
 'up': 14352,
 'super': 2727,
 'left': 4958,
 'portions': 209,
 'im': 7768,
 'into': 6364,
 'this': 83593,
 'exactly': 4678,
 'lunch': 2407,
 'enough': 5231,
 'added': 19387,
 'okay': 717,
 'whole': 5540,
 'only': 13679,
 'though': 4791,
 'pounds': 275,
 'a': 84192,
 'frozen': 2722,
 'fillet': 86,
 'so': 39441,
 'dish': 8447,
 'those': 2408,
 'food': 3473,
 'in': 43940,
 'was': 56972,
 'out': 22223,
 'weight': 290,
 'me': 9996,
 'the': 95894,
 'on': 28791,
 'to': 72118,
 'over': 8762,
 'white': 3493,
 'for': 75829,
 'change': 4390,
 'local': 565,
 'sale': 255,
 'cut': 6416,
 'yummy': 6943,
 'bu

In [237]:
get_by_bad == get_by_good

True

## **Сравнение**

In [231]:
%%timeit 
get_word_reviews_count(reviews)

26.6 s ± 168 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [238]:
%%timeit 
get_word_reviews_count_opt(reviews)

3.76 s ± 391 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## **Вывод**

Время уменьшилось в $\sim 7$ раз